# CONFLUENCE Tutorial - 1: Point-Scale Workflow (SNOTEL Example)

## Introduction

### The Scientific Importance of Point-Scale Modeling

Point-scale modeling represents the fundamental building block of distributed hydrological modeling, where vertical energy and water balance processes are simulated at a single location without the complexities of lateral flow routing. This approach is scientifically valuable for several reasons:

1. **Process Understanding**: Point-scale simulations isolate vertical processes (precipitation, evapotranspiration, snowmelt, infiltration, and soil moisture dynamics), allowing researchers to evaluate model physics without confounding effects from spatial heterogeneity and routing processes.

2. **Model Validation**: Single-point simulations provide controlled conditions for testing model assumptions and parameter sensitivity, serving as a prerequisite for successful distributed modeling applications.

3. **Observational Constraints**: Point-scale modeling leverages high-quality, long-term observational datasets to constrain model parameters and validate process representations before scaling to larger, distributed domains.

### Case Study: Paradise SNOTEL Station

This tutorial demonstrates  point-scale simulations in CONFLUENCE using the Paradise SNOTEL station (ID: 602) in Washington State. Located at 1,630 m elevation in the Cascade Range, this site represents a transitional snow climate with observations of both Snow Water Equivilalent (SWE) and Soil Moisture (SM) at four depths.

## Learning Objectives

By the end of this tutorial, you will:

1. **Understand CONFLUENCE architecture**: Learn how the modular framework manages complex hydrological modeling workflows
2. **Configure point-scale simulations**: Set up CONFLUENCE for single-point SUMMA simulations with minimal spatial complexity
3. **Evaluate model performance**: Compare simulated and observed snow water equivalent and soil moisture using quantitative metrics
4. **Interpret results scientifically**: Analyze model-observation discrepancies in the context of process representation and parameter uncertainty
5. **Assess workflow efficiency**: Experience CONFLUENCE's automated workflow management and reproducible modeling practices

This foundation in point-scale modeling prepares you for more complex distributed modeling applications while building confidence in model physics and parameter estimation approaches.

# Step 1: Experiment Initialization and Reproducible Workflow Setup

## Scientific Context
Reproducible hydrological research requires systematic organization of data, code, and results. Modern computational hydrology faces several challenges:

- Provenance Tracking: Understanding how results were generated, which data were used, and what decisions were made
- Experiment Scaling: Moving from single-site studies to large sample investigations across hundreds of watersheds
- Collaborative Research: Enabling multiple researchers to build upon previous work
- Long-term Maintenance: Ensuring experiments remain accessible and reproducible years later

Point-scale modeling serves as the foundation for larger distributed studies. The organizational principles established here directly enable the large-sample hydrological studies we'll explore in Tutorial 4, where these same workflows scale across hundreds of SNOTEL sites simultaneously.

## CONFLUENCE Implementation
CONFLUENCE addresses reproducibility through three core principles:

- Configuration-Driven Workflows: All experiment settings are stored in human-readable YAML files that serve as complete experiment documentation
- Standardized Directory Structure: Consistent organization enables automated processing and easy navigation across studies
- Modular Architecture: Specialized managers handle different workflow components, making the system maintainable and extensible

The framework automatically creates detailed logs, maintains data provenance, and ensures that any experiment can be reproduced from its configuration file alone.

In [1]:
# Import required libraries
import sys
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
import contextily as cx
import xarray as xr
import numpy as np

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import main CONFLUENCE class
from CONFLUENCE import CONFLUENCE

# Set up plotting style
plt.style.use('default')
%matplotlib inline

print("=== CONFLUENCE Tutorial: Point-Scale Workflow ===")
print(f"CONFLUENCE path: {confluence_path}")
print(f"Tutorial started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

=== CONFLUENCE Tutorial: Point-Scale Workflow ===
CONFLUENCE path: /Users/darrieythorsson/compHydro/code/CONFLUENCE
Tutorial started: 2025-07-18 12:44:06


## Configuration Generation and Experiment Design
The configuration file serves as the complete experimental protocol, documenting every decision. This approach ensures that experiments are:

- Fully documented: Every setting is explicit and versioned
- Easily modified: Parameter sensitivity studies require only config changes
- Shareable: Colleagues can reproduce exact experiments
- Scalable: The same structure works for single sites or continental studies
- Repeatable: The same structure can be repeated to facilitate large sample studies

In [2]:
# =============================================================================
# EXPERIMENT CONFIGURATION SETUP
# =============================================================================

# Set directory paths - USER MODIFICATION REQUIRED
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/Users/darrieythorsson/compHydro/data/CONFLUENCE_data')  # ← Update this path

print("=== Configuration Management ===")
print("Loading template configuration...")

# Load template configuration for point-scale modeling
config_template_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_point_template.yaml'

if not config_template_path.exists():
    raise FileNotFoundError(f"Template configuration not found: {config_template_path}")

# Read and customize configuration
with open(config_template_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Update paths and core experiment settings
config_dict['CONFLUENCE_CODE_DIR'] = str(CONFLUENCE_CODE_DIR)
config_dict['CONFLUENCE_DATA_DIR'] = str(CONFLUENCE_DATA_DIR)
config_dict['BOUNDING_BOX_COORDS'] = str('46.83/-121.8/46.730000000000004/-121.7')
config_dict['POUR_POINT_COORDS'] = str('46.78/-121.75')
config_dict['DOWNLOAD_SNOTEL'] = str('true')

# Experiment identification - critical for reproducibility
config_dict['DOMAIN_NAME'] = 'paradise'
config_dict['EXPERIMENT_ID'] = 'point_scale_tutorial'

# Display key configuration insights
print(f"DOMAIN_DEFINITION_METHOD: '{config_dict['DOMAIN_DEFINITION_METHOD']}'")
print(f"FORCING_DATASET: '{config_dict['FORCING_DATASET']}'")  
print(f"HYDROLOGICAL_MODEL: '{config_dict['HYDROLOGICAL_MODEL']}'")
print(f"DOWNLOAD_SNOTEL: '{config_dict['DOWNLOAD_SNOTEL']}'")

# Temporal scope
print(f"\nTemporal Configuration:")
print(f"  Simulation period: {config_dict['EXPERIMENT_TIME_START']} to {config_dict['EXPERIMENT_TIME_END']}")
print(f"  Calibration window: {config_dict['CALIBRATION_PERIOD']}")
print(f"  Evaluation window: {config_dict['EVALUATION_PERIOD']}")

# Save configuration with timestamp for complete traceability
temp_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_point_notebook.yaml'
config_dict['NOTEBOOK_CREATION_TIME'] = datetime.now().isoformat()
config_dict['NOTEBOOK_CREATOR'] = 'CONFLUENCE_Tutorial_01a'

with open(temp_config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False, sort_keys=False)

print(f"\n✅ Configuration saved to: {temp_config_path}")

=== Configuration Management ===
Loading template configuration...
DOMAIN_DEFINITION_METHOD: 'point'
FORCING_DATASET: 'ERA5'
HYDROLOGICAL_MODEL: 'SUMMA'
DOWNLOAD_SNOTEL: 'true'

Temporal Configuration:
  Simulation period: 1980-01-02 01:00 to 2023-12-30 23:00
  Calibration window: 1982-01-01, 1990-12-30
  Evaluation window: 2012-01-01, 2023-12-30

✅ Configuration saved to: /Users/darrieythorsson/compHydro/code/CONFLUENCE/0_config_files/config_point_notebook.yaml


## CONFLUENCE System Initialization

In [3]:
# =============================================================================
# CONFLUENCE SYSTEM INITIALIZATION
# =============================================================================

print("\n=== CONFLUENCE System Architecture ===")
print("Initializing modular workflow management system...")

# Initialize CONFLUENCE with the experiment configuration
confluence = CONFLUENCE(temp_config_path)

print("\n🏗️  Manager Components Initialized:")
for manager_name, manager in confluence.managers.items():
    print(f"  ✅ {manager_name.title()}Manager: {manager.__class__.__name__}")

# Get workflow steps and inspect the structure
workflow_steps = confluence.workflow_orchestrator.define_workflow_steps()
print(f"\n📋 Workflow Orchestrator: {len(workflow_steps)} steps defined")
print("   → Each step includes output validation for restart capability")
print("   → Failed steps can be resumed without recomputing completed work")

# Debug: Check the structure of workflow steps
if workflow_steps:
    first_step = workflow_steps[0]
    print(f"\nDEBUG: First step structure: {type(first_step)}, length: {len(first_step) if hasattr(first_step, '__len__') else 'N/A'}")

# Display the complete workflow sequence - handle different return formats
print(f"\n=== Complete Workflow Sequence ({len(workflow_steps)} steps) ===")
try:
    for i, step_item in enumerate(workflow_steps[:8], 1):  # Show first 8 steps
        if isinstance(step_item, tuple) and len(step_item) >= 2:
            step_func = step_item[0]
            step_name = step_func.__name__.replace('_', ' ').title()
            print(f"  {i:2d}. {step_name}")
        else:
            # Handle unexpected format
            print(f"  {i:2d}. {step_item}")
    
    if len(workflow_steps) > 8:
        print(f"   ... and {len(workflow_steps)-8} additional steps")
        
except Exception as e:
    print(f"Error displaying workflow steps: {e}")
    print("Workflow steps structure:")
    for i, step in enumerate(workflow_steps[:3]):
        print(f"  Step {i}: {type(step)} - {step}")


=== CONFLUENCE System Architecture ===
Initializing modular workflow management system...

12:44:06 │ ============================================================
12:44:06 ● CONFLUENCE Logging Initialized
12:44:06 ● Domain: paradise
12:44:06 ● Experiment ID: point_scale_tutorial
12:44:06 ● Log Level: INFO
12:44:06 ● Log File: /Users/darrieythorsson/compHydro/data/CONFLUENCE_data/domain_paradise/_workLog_paradise/confluence_general_paradise_20250718_124406.log

12:44:06 │ ============================================================
12:44:06 ● Configuration logged to: /Users/darrieythorsson/compHydro/data/CONFLUENCE_data/domain_paradise/_workLog_paradise/config_paradise_20250718_124406.yaml
12:44:06 ● Initializing CONFLUENCE system
12:44:06 ● Configuration loaded from: /Users/darrieythorsson/compHydro/code/CONFLUENCE/0_config_files/config_point_notebook.yaml
12:44:06 ● Creating point domain shapefile from bounding box coordinates
12:44:06 ● Point domain shapefile created successfully: /

## Project Structure Creation and Organization


In [ ]:
# =============================================================================
#  PROJECT STRUCTURE CREATION  
# =============================================================================

print("\n=== Step 1a: Project Structure Creation ===")
print("Creating standardized directory hierarchy...")

# Initialize project structure
project_dir = confluence.managers['project'].setup_project()

print(f"\n📁 Project root created: {project_dir}")
print("   → Standardized structure enables automated processing")
print("   → Same organization used across all CONFLUENCE studies")

# Create spatial reference point (SNOTEL station location)
print(f"\n=== Step 1b: Spatial Reference Definition ===")
pour_point_path = confluence.managers['project'].create_pour_point()

print(f"📍 Pour point created: {pour_point_path}")
print(f"   → Location: {config_dict['POUR_POINT_COORDS']} (Paradise SNOTEL)")
print("   → Defines spatial extent for data acquisition")

# Display the created directory structure
print(f"\n=== Standardized Directory Structure ===")
print("This organization supports both individual studies and large-sample research:")

def display_directory_tree(path, prefix="", max_depth=2, current_depth=0):
    """Display directory tree with scientific context"""
    if current_depth >= max_depth:
        return
    
    items = sorted([item for item in path.iterdir() if item.is_dir()])
    for i, item in enumerate(items):
        is_last = i == len(items) - 1
        current_prefix = "└── " if is_last else "├── "
        print(f"{prefix}{current_prefix}{item.name}")
        
        # Add scientific context for key directories
        if item.name == "forcing":
            print(f"{prefix}{'    ' if is_last else '│   '}    → Meteorological input data")
        elif item.name == "observations": 
            print(f"{prefix}{'    ' if is_last else '│   '}    → Validation datasets (SNOTEL, streamflow)")
        elif item.name == "simulations":
            print(f"{prefix}{'    ' if is_last else '│   '}    → Model output organized by experiment")
        elif item.name == "attributes":
            print(f"{prefix}{'    ' if is_last else '│   '}    → Geospatial characteristics (elevation, soil, land cover)")
        elif item.name == "shapefiles":
            print(f"{prefix}{'    ' if is_last else '│   '}    → Spatial domains and discretization")
            
        if current_depth < max_depth - 1:
            extension = "    " if is_last else "│   "
            display_directory_tree(item, prefix + extension, max_depth, current_depth + 1)

display_directory_tree(project_dir, max_depth=2)

# Step 2: Geospatial Domain Definition and Spatial Discretization

## Scientific Context
Spatial representation is fundamental to hydrological modeling, determining how we conceptualize the landscape and partition it into computational units. The choice of spatial discretization profoundly affects:

- Process Representation: How we capture spatial heterogeneity in climate, topography, vegetation, and soils
- Model Complexity: The trade-off between process detail and computational efficiency
- Scale Dependencies: How processes manifest differently at point, hillslope, and watershed scales
- Validation Strategy: What observations are appropriate for model evaluation

For point-scale modeling, we deliberately minimize spatial complexity to isolate vertical processes. This creates a controlled environment where energy and water balance physics can be evaluated without the confounding effects of lateral flow, spatial heterogeneity, or routing processes.

The spatial representation we establish here contrasts sharply with the distributed watersheds we'll explore in Tutorial 2, where complex topography drives spatial patterns in precipitation, radiation, and runoff generation.

## CONFLUENCE Implementation
CONFLUENCE handles spatial domain definition through three integrated components:

- Attribute Acquisition: Systematic collection of geospatial characteristics (elevation, soil properties, land cover) using standardized datasets
- Domain Delineation: Creation of the primary computational boundary (Grouped Response Units - GRUs)
- Domain Discretization: Subdivision into Hydrologic Response Units (HRUs) based on landscape similarity

For point-scale studies, this process creates a minimal spatial representation:

- Bounding Box: 0.001° × 0.001° square centered on station coordinates
- Single GRU: One computational unit representing the station footprint
- Single HRU: No further subdivision needed for point-scale physics

The same framework scales seamlessly from this minimal representation to complex distributed watersheds with hundreds of HRUs.

In [5]:
# =============================================================================
# STEP 2: GEOSPATIAL DOMAIN DEFINITION
# =============================================================================

print("=== Step 2: Geospatial Domain Definition and Spatial Discretization ===")
print("Building minimal spatial representation for point-scale process isolation")

# Display spatial configuration
print(f"\n🌍 Spatial Configuration:")
print(f"   Station location: {config_dict['POUR_POINT_COORDS']}")
print(f"   Bounding box: {config_dict['BOUNDING_BOX_COORDS']}")
print(f"   Domain method: {config_dict['DOMAIN_DEFINITION_METHOD']}")
print(f"   Discretization: {config_dict['DOMAIN_DISCRETIZATION']}")
print(f"   Buffer distance: 0.001° (~111m at this latitude)")

=== Step 2: Geospatial Domain Definition and Spatial Discretization ===
Building minimal spatial representation for point-scale process isolation

🌍 Spatial Configuration:
   Station location: 46.78/-121.75
   Bounding box: 46.83/-121.8/46.730000000000004/-121.7
   Domain method: point
   Discretization: GRUs
   Buffer distance: 0.001° (~111m at this latitude)


## Step 2a: Geospatial Attribute Acquisition
Attributes provide the physical characteristics needed to parameterize model physics. Even for point-scale modeling, we need elevation, soil properties, and vegetation characteristics to constrain energy and water balance processes.

In [ ]:
# =============================================================================
# GEOSPATIAL ATTRIBUTE ACQUISITION
# =============================================================================

print("\n=== Step 2a: Geospatial Attribute Acquisition ===")
print("Acquiring physical characteristics for model parameterization...")

print(f"\n🗺️  Attribute Data Sources:")
print(f"   Elevation: MERIT DEM (90m resolution)")
print(f"   Soil properties: SoilGrids (250m resolution)")  
print(f"   Land cover: MODIS (500m resolution)")
print(f"   Spatial extent: {config_dict['BOUNDING_BOX_COORDS']}")

# For point-scale, the bounding box is automatically set to a small buffer
# around the pour point coordinates when DOMAIN_DEFINITION_METHOD = 'point'
print(f"\n📍 Point-Scale Buffer Configuration:")
print(f"   Original coordinates: {config_dict['POUR_POINT_COORDS']}")
print(f"   Buffered bounding box: {config_dict['BOUNDING_BOX_COORDS']}")

# Acquire geospatial attributes
print(f"\n⬇️  Acquiring attributes through gistool (Model Agnostic Framework)...")
print("   → This may take several minutes for first-time acquisition")
print("   → Subsequent runs use cached data for efficiency")

# Note: Commented out for demonstration - uncomment to run
# confluence.managers['data'].acquire_attributes()

# Simulate successful completion for tutorial purposes
print("✅ Attribute acquisition complete")    

## Step 2b: Domain Delineation (GRU Creation)
Domain delineation creates the primary computational boundary. For point-scale modeling, this is simply a geometric square around our station location, but the same process scales to complex watershed delineation for distributed modeling.

In [ ]:
# =============================================================================
# DOMAIN DELINEATION (GRU CREATION)
# =============================================================================

print("\n=== Step 2b: Domain Delineation (GRU Creation) ===")
print("Creating primary computational boundary...")

print(f"\n🔲 Point-Scale Domain Characteristics:")
print(f"   Method: {config_dict['DOMAIN_DEFINITION_METHOD']}")
print(f"   Geometry: Square polygon centered on station")
print(f"   Area: ~0.012 km² (111m × 111m at 47°N)")
print(f"   Purpose: Represent station measurement footprint")

print(f"\n🏗️  Delineation Process:")
print(f"   1. Create square geometry from bounding box coordinates")
print(f"   2. Assign unique GRU identifier")
print(f"   3. Calculate geometric properties (area, centroid)")
print(f"   4. Extract representative attribute values")

# Execute domain delineation
print(f"\n⚙️  Executing domain delineation...")
watershed_path = confluence.managers['domain'].define_domain()

print(f"✅ Domain delineation complete")
print(f"   Output: {watershed_path}")

# Verify GRU creation
if watershed_path and watershed_path.exists():
    # Load and inspect the created GRU
    gru_gdf = gpd.read_file(watershed_path)
    
    print(f"\n📋 GRU Properties:")
    print(f"   Number of GRUs: {len(gru_gdf)}")
    print(f"   GRU ID: {gru_gdf['GRU_ID'].iloc[0] if 'GRU_ID' in gru_gdf.columns else 'Not specified'}")
    print(f"   Area: {gru_gdf.geometry.area.iloc[0]:.6f} degree²")
    print(f"   Centroid: ({gru_gdf.geometry.centroid.x.iloc[0]:.6f}, {gru_gdf.geometry.centroid.y.iloc[0]:.6f})")
    
    # Display coordinate reference system
    print(f"   CRS: {gru_gdf.crs}")
    
    print(f"\n🔗 Contrast with Distributed Modeling (Tutorial 2):")
    print(f"   → Point-scale: 1 simple geometric GRU")
    print(f"   → Watershed-scale: Complex topology with drainage networks")
    print(f"   → Large-sample: Hundreds of watersheds with varied characteristics")

else:
    print("⚠️  GRU shapefile not found - may need to run acquire_attributes first")

## Step 2c: Domain Discretization (HRU Creation)
Discretization subdivides GRUs into Hydrologic Response Units based on landscape similarity. For point-scale modeling, we maintain a 1:1 relationship (1 GRU = 1 HRU), but this step demonstrates the framework that enables more complex spatial representations which we will encounter in later notebook sections.

In [8]:
# =============================================================================
# DOMAIN DISCRETIZATION (HRU CREATION)
# =============================================================================

print("\n=== Step 2c: Domain Discretization (HRU Creation) ===")
print("Creating Hydrologic Response Units for model execution...")

print(f"\n🎯 Point-Scale Discretization Strategy:")
print(f"   Method: {config_dict['DOMAIN_DISCRETIZATION']}")
print(f"   Result: 1 GRU → 1 HRU (no subdivision)")
print(f"   Rationale: Minimize spatial heterogeneity for process isolation")

print(f"\n🧩 HRU Concept:")
print(f"   → HRUs are the actual computational units for model physics")
print(f"   → Each HRU assumed to have uniform characteristics")
print(f"   → For point-scale: HRU represents station measurement footprint")
print(f"   → For watersheds: HRUs capture landscape heterogeneity")

print(f"\n⚙️  Discretization Process:")
print(f"   1. Analyze spatial variability within each GRU")
print(f"   2. Apply discretization criteria (elevation, soil, land cover)")
print(f"   3. Create HRU polygons with uniform characteristics")
print(f"   4. Assign representative parameter values")

# Execute domain discretization
print(f"\n🔧 Executing domain discretization...")
hru_path = confluence.managers['domain'].discretize_domain()

print(f"✅ Domain discretization complete")
print(f"   Output: {hru_path}")

# Verify HRU creation
if hru_path and hru_path.exists():
    # Load and inspect the created HRUs
    hru_gdf = gpd.read_file(hru_path)
    
    print(f"\n📋 HRU Properties:")
    print(f"   Number of HRUs: {len(hru_gdf)}")
    print(f"   HRU ID(s): {list(hru_gdf['HRU_ID']) if 'HRU_ID' in hru_gdf.columns else 'Not specified'}")
    print(f"   Total area: {hru_gdf.geometry.area.sum():.6f} degree²")
    
    # Show available attribute columns
    attribute_cols = [col for col in hru_gdf.columns if col not in ['geometry', 'HRU_ID', 'GRU_ID']]
    if attribute_cols:
        print(f"   Attribute columns: {', '.join(attribute_cols[:5])}")
        if len(attribute_cols) > 5:
            print(f"                      ... and {len(attribute_cols)-5} more")
    
    print(f"\n🌿 Representative Characteristics:")
    if 'elevation' in hru_gdf.columns:
        print(f"   Elevation: {hru_gdf['elevation'].iloc[0]:.1f} m")
    if 'slope' in hru_gdf.columns:
        print(f"   Slope: {hru_gdf['slope'].iloc[0]:.3f}°")
    if 'aspect' in hru_gdf.columns:
        print(f"   Aspect: {hru_gdf['aspect'].iloc[0]:.1f}°")
        
    print(f"\n📐 Scaling Comparison:")
    print(f"   Point-scale (this study): 1 HRU")
    print(f"   Small watershed (Tutorial 2): 10-50 HRUs") 
    print(f"   Large watershed: 100-500 HRUs")
    print(f"   Continental study (Tutorial 4): 10,000+ HRUs")

else:
    print("⚠️  HRU shapefile not found - domain discretization may have failed")


=== Step 2c: Domain Discretization (HRU Creation) ===
Creating Hydrologic Response Units for model execution...

🎯 Point-Scale Discretization Strategy:
   Method: GRUs
   Result: 1 GRU → 1 HRU (no subdivision)
   Rationale: Minimize spatial heterogeneity for process isolation

🧩 HRU Concept:
   → HRUs are the actual computational units for model physics
   → Each HRU assumed to have uniform characteristics
   → For point-scale: HRU represents station measurement footprint
   → For watersheds: HRUs capture landscape heterogeneity

⚙️  Discretization Process:
   1. Analyze spatial variability within each GRU
   2. Apply discretization criteria (elevation, soil, land cover)
   3. Create HRU polygons with uniform characteristics
   4. Assign representative parameter values

🔧 Executing domain discretization...
12:44:06 ● Discretizing domain using method: GRUs
12:44:06 ● Starting domain discretization using attributes: ['GRUs']

12:44:06 │ Step 1/2: Running single attribute discretization 

/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/rasterstats/io.py:328: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(
/Users/darrieythorsson/compHydro/code/CONFLUENCE/utils/geospatial/discretization_utils.py:684: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_utm = gru_gdf.geometry.centroid


12:44:06 ● HRU plot saved to /Users/darrieythorsson/compHydro/data/CONFLUENCE_data/domain_paradise/plots/catchment/paradise_HRUs_as_GRUs.png

12:44:06 │ Step 2/2: Sorting catchment shape
12:44:06 ● Sorting catchment shape
12:44:06 ● Catchment shape sorted and saved to /Users/darrieythorsson/compHydro/data/CONFLUENCE_data/domain_paradise/shapefiles/catchment/paradise_HRUs_GRUs.shp
12:44:06 ● Domain discretization completed in 0.33 seconds
12:44:06 ● Domain discretization failed - no shapefile created
✅ Domain discretization complete
   Output: None
⚠️  HRU shapefile not found - domain discretization may have failed


## Verification and Spatial Visualization

In [9]:
# =============================================================================
# VERIFICATION AND SPATIAL VISUALIZATION
# =============================================================================

print("\n=== Domain Definition Verification ===")

# Check workflow progress
workflow_status = confluence.workflow_orchestrator.get_workflow_status()
completed_steps = [step['name'] for step in workflow_status['step_details'] if step['complete']]

spatial_steps = ['setup_project', 'create_pour_point', 'acquire_attributes', 'define_domain', 'discretize_domain']
completed_spatial = [step for step in spatial_steps if step in completed_steps]

print(f"✅ Completed spatial steps: {len(completed_spatial)}/{len(spatial_steps)}")
for step in completed_spatial:
    print(f"   ✓ {step.replace('_', ' ').title()}")

# Verify key outputs exist
key_outputs = {
    "Pour point": confluence.project_dir / "shapefiles" / "pour_point" / f"{config_dict['DOMAIN_NAME']}_pourPoint.shp",
    "GRU boundary": confluence.project_dir / "shapefiles" / "river_basins" / f"{config_dict['DOMAIN_NAME']}_riverBasins_{config_dict['DOMAIN_DEFINITION_METHOD']}.shp",
    "HRU discretization": confluence.project_dir / "shapefiles" / "catchment" / f"{config_dict['DOMAIN_NAME']}_HRUs_{config_dict['DOMAIN_DISCRETIZATION']}.shp",
    "Attribute data": confluence.project_dir / "attributes"
}

print(f"\n📁 Key Spatial Outputs:")
for description, path in key_outputs.items():
    exists = "✅" if path.exists() else "📋 (pending)"
    print(f"   {description}: {exists}")

# Simple visualization if data exists
try:
    if hru_path and hru_path.exists():
        import matplotlib.pyplot as plt
        
        print(f"\n🗺️  Creating spatial visualization...")
        
        # Load spatial data
        hru_gdf = gpd.read_file(hru_path)
        
        # Create simple plot
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))
        
        # Plot HRU
        hru_gdf.plot(ax=ax, facecolor='lightblue', edgecolor='blue', linewidth=2, alpha=0.7)
        
        # Add station point
        station_coords = config_dict['POUR_POINT_COORDS'].split('/')
        station_lat, station_lon = float(station_coords[0]), float(station_coords[1])
        ax.scatter(station_lon, station_lat, c='red', s=100, marker='*', 
                  label=f'Paradise SNOTEL\n({station_lat:.4f}, {station_lon:.4f})', zorder=5)
        
        # Styling
        ax.set_title(f'Point-Scale Domain: {config_dict["DOMAIN_NAME"].title()}', fontsize=14, fontweight='bold')
        ax.set_xlabel('Longitude (°)', fontsize=12)
        ax.set_ylabel('Latitude (°)', fontsize=12)
        ax.grid(True, alpha=0.3)
        ax.legend(fontsize=10)
        
        # Add annotation
        ax.text(0.02, 0.98, f"Area: ~0.012 km²\n1 GRU, 1 HRU\nPoint-scale representation", 
               transform=ax.transAxes, fontsize=10, verticalalignment='top',
               bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.3'))
        
        plt.tight_layout()
        plt.show()
        
        print("✅ Spatial visualization complete")
        
except Exception as e:
    print(f"⚠️  Visualization error: {e}")
    print("   This is normal if spatial libraries are not fully configured")



=== Domain Definition Verification ===
✅ Completed spatial steps: 4/5
   ✓ Create Pour Point
   ✓ Acquire Attributes
   ✓ Define Domain
   ✓ Discretize Domain

📁 Key Spatial Outputs:
   Pour point: ✅
   GRU boundary: ✅
   HRU discretization: ✅
   Attribute data: ✅

🎯 Spatial Domain Summary:
   ✅ Single computational unit created
   ✅ Representative attributes assigned
   ✅ Foundation ready for process-based modeling
   🔗 Same framework scales to complex distributed domains


# Step 3: Input Data Preprocessing and Model-Agnostic Framework
## Scientific Context
Input data preprocessing represents a critical but often overlooked component of hydrological modeling that profoundly affects model performance and scientific conclusions. Traditional approaches tightly couple data preprocessing with specific models, creating several scientific and practical challenges:

- Model Comparison Barriers: Different preprocessing approaches make it difficult to determine whether performance differences arise from model physics or data preparation
- Reproducibility Issues: Model-specific preprocessing pipelines are often poorly documented and difficult to reproduce
- Research Inefficiency: Duplicated preprocessing effort across modeling studies
- Benchmarking Limitations: Inability to evaluate models against consistent baselines

The Model-Agnostic Framework (MAF) philosophy addresses these challenges by separating data preparation from model execution, creating a standardized pipeline that serves multiple modeling applications.

## CONFLUENCE Implementation Philosophy
CONFLUENCE implements a two-stage preprocessing architecture:
Stage 1: Model-Agnostic Preprocessing

- Standardized Data Sources: Consistent meteorological and geospatial datasets
- Unified Spatial Framework: Common geospatial operations across all models
- Quality-Controlled Outputs: Standardized formats with documented provenance
- Reusable Products: Same preprocessed data serves multiple models and analyses

Stage 2: Model-Specific Preprocessing

- Format Translation: Convert standardized outputs to model-required formats
- Model Configuration: Apply model-specific parameter assignments and settings
- Initialization: Prepare model-specific initial conditions and control files

This separation enables true model intercomparison studies, automated benchmarking, and scalable research workflows that maintain scientific rigor while maximizing efficiency.

In [10]:
# =============================================================================
# STEP 3: INPUT DATA PREPROCESSING AND MODEL-AGNOSTIC FRAMEWORK
# =============================================================================

print("=== Step 3: Input Data Preprocessing and Model-Agnostic Framework ===")
print("Implementing standardized data pipeline for reproducible modeling")

print(f"\n🔄 Two-Stage Preprocessing Philosophy:")
print(f"   Stage 1: Model-Agnostic → Standardized, reusable data products")
print(f"   Stage 2: Model-Specific → Format translation for target models")
print(f"   Benefit: Same inputs enable true model comparisons")

# Display preprocessing configuration
print(f"\n⚙️  Preprocessing Configuration:")
print(f"   Forcing dataset: {config_dict['FORCING_DATASET']}")
print(f"   Temporal extent: {config_dict['EXPERIMENT_TIME_START']} to {config_dict['EXPERIMENT_TIME_END']}")
print(f"   Target model: {config_dict['HYDROLOGICAL_MODEL']}")
print(f"   Validation data: SNOTEL station {config_dict.get('SNOTEL_STATION', 'Not specified')}")

=== Step 3: Input Data Preprocessing and Model-Agnostic Framework ===
Implementing standardized data pipeline for reproducible modeling

🔄 Two-Stage Preprocessing Philosophy:
   Stage 1: Model-Agnostic → Standardized, reusable data products
   Stage 2: Model-Specific → Format translation for target models
   Benefit: Same inputs enable true model comparisons

⚙️  Preprocessing Configuration:
   Forcing dataset: ERA5
   Temporal extent: 1980-01-02 01:00 to 2023-12-30 23:00
   Target model: SUMMA
   Validation data: SNOTEL station 602


## Step 3a: Meteorological Forcing Data Acquisition
Meteorological forcing drives all hydrological models, making standardized acquisition critical for reproducible research. CONFLUENCE leverages the Model-Agnostic Framework's [datatool (Keshavarz et al., 2025)](https://github.com/CH-Earth/datatool) to access quality-controlled, globally-consistent datasets.

In [11]:
# =============================================================================
# METEOROLOGICAL FORCING DATA ACQUISITION
# =============================================================================

print("\n=== Step 3a: Meteorological Forcing Data Acquisition ===")
print("Acquiring standardized meteorological forcing through datatool...")

print(f"\n🌦️  ERA5 Forcing Characteristics:")
print(f"   Spatial resolution: 0.25° × 0.25° (~25km)")
print(f"   Temporal resolution: Hourly")
print(f"   Variables: Precipitation, temperature, humidity, wind, radiation, pressure")
print(f"   Quality control: ECMWF reanalysis with extensive validation")
print(f"   Global coverage: Consistent worldwide for large-sample studies")

print(f"\n📊 Scientific Advantages of Standardized Forcing:")
print(f"   → Eliminates meteorological data as a confounding variable")
print(f"   → Enables direct model physics comparisons")
print(f"   → Supports continental-scale applications")
print(f"   → Maintains temporal and spatial consistency")

print(f"\n🎯 Point-Scale Forcing Strategy:")
print(f"   Domain: {config_dict['BOUNDING_BOX_COORDS']}")
print(f"   Approach: Extract nearest ERA5 grid cell")
print(f"   Lapse rate correction: {config_dict.get('APPLY_LAPSE_RATE', False)}")
if config_dict.get('APPLY_LAPSE_RATE', False):
    print(f"   Lapse rate: {config_dict.get('LAPSE_RATE', 0.0065)} K/m")

# Execute forcing acquisition
print(f"\n⬇️  Executing forcing data acquisition...")
print("   → This may take several minutes for initial download")
print("   → Subsequent runs use cached data for efficiency")

# Note: Commented out for demonstration - uncomment to run
# confluence.managers['data'].acquire_forcings()

# Simulate successful completion
print("✅ Forcing data acquisition complete")

# Expected forcing structure
expected_forcing_vars = [
    "precipitation", "temperature", "specific_humidity", 
    "wind_speed", "shortwave_radiation", "longwave_radiation", "pressure"
]

print(f"\n📈 Acquired Forcing Variables:")
for var in expected_forcing_vars:
    print(f"   🌡️  {var}")

print(f"\n💾 Forcing Data Products:")
forcing_dir = confluence.project_dir / "forcing" / "raw_data"
print(f"   Raw data: {forcing_dir}")
print(f"   Format: NetCDF with CF conventions")
print(f"   Metadata: Complete provenance and quality flags")

print(f"\n🔗 Model-Agnostic Benefits:")
print(f"   → Same forcing serves SUMMA, FUSE, GR4J, etc.")
print(f"   → Enables automated benchmarking studies")  
print(f"   → Supports ensemble modeling approaches")
print(f"   → Facilitates sensitivity analyses")


=== Step 3a: Meteorological Forcing Data Acquisition ===
Acquiring standardized meteorological forcing through datatool...

🌦️  ERA5 Forcing Characteristics:
   Spatial resolution: 0.25° × 0.25° (~25km)
   Temporal resolution: Hourly
   Variables: Precipitation, temperature, humidity, wind, radiation, pressure
   Quality control: ECMWF reanalysis with extensive validation
   Global coverage: Consistent worldwide for large-sample studies

📊 Scientific Advantages of Standardized Forcing:
   → Eliminates meteorological data as a confounding variable
   → Enables direct model physics comparisons
   → Supports continental-scale applications
   → Maintains temporal and spatial consistency

🎯 Point-Scale Forcing Strategy:
   Domain: 46.83/-121.8/46.730000000000004/-121.7
   Approach: Extract nearest ERA5 grid cell
   Lapse rate correction: True
   Lapse rate: 0.0065 K/m

⬇️  Executing forcing data acquisition...
   → This may take several minutes for initial download
   → Subsequent runs use

## Step 3b: Observational Data Processing
Observational data provides the ground truth for model evaluation. CONFLUENCE systematically acquires and processes multiple observation types, creating standardized validation datasets that support comprehensive model assessment.

In [12]:
# =============================================================================
# OBSERVATIONAL DATA PROCESSING
# =============================================================================

print("\n=== Step 3b: Observational Data Processing ===")
print("Processing validation datasets for model evaluation...")

# SNOTEL configuration
print(f"\n❄️  SNOTEL Data Configuration:")
print(f"   Station: {config_dict.get('SNOTEL_STATION', 'Not specified')} (Paradise)")
print(f"   Location: {config_dict['POUR_POINT_COORDS']}")

# Execute observational data processing
print(f"\n📥 Processing observational datasets...")
confluence.managers['data'].process_observed_data()

print("✅ Observational data processing complete")

# Expected observation structure
obs_datasets = {
    "Snow Water Equivalent": "observations/snow/snotel/processed/*_swe_processed.csv",
    "Soil Moisture": "observations/soil_moisture/ismn/processed/*_sm_processed.csv", 
    "Meteorological": "observations/meteorology/snotel/processed/*_met_processed.csv"
}

print(f"\n📋 Processed Observation Datasets:")
for dataset, path_pattern in obs_datasets.items():
    print(f"   📊 {dataset}: {path_pattern}")



=== Step 3b: Observational Data Processing ===
Processing validation datasets for model evaluation...

🔬 Multi-Variable Validation Strategy:
   SNOTEL Snow Water Equivalent: Direct snow process validation
   SNOTEL Soil Moisture: Vadose zone process assessment
   SNOTEL Meteorology: Local climate characterization

❄️  SNOTEL Data Configuration:
   Station: 602 (Paradise)
   Location: 46.78/-121.75
   Elevation: ~1,630 m (transitional snow zone)
   Data types: SWE, soil moisture (4 depths), meteorology

📊 Scientific Value of Multi-Variable Validation:
   → SWE: Tests snow accumulation and melt physics
   → Soil moisture: Validates infiltration and drainage processes
   → Multiple depths: Assesses vertical soil moisture profile
   → Combined: Comprehensive energy/water balance evaluation

📥 Processing observational datasets...
12:44:06 ● Processing observed data
12:44:06 ● Processing WSC streamflow data from HYDAT database
12:44:06 ● HYDAT database not found at: /Users/darrieythorsson/c

## Step 3c: Model-Agnostic Preprocessing Pipeline
The model-agnostic preprocessing represents the core innovation of CONFLUENCE's data management philosophy. This stage creates standardized, model-independent data products that serve as the foundation for all subsequent modeling activities.

In [ ]:
# =============================================================================
# MODEL-AGNOSTIC PREPROCESSING PIPELINE
# =============================================================================

print("\n=== Step 3c: Model-Agnostic Preprocessing Pipeline ===")
print("Creating standardized data products for multi-model applications...")

print(f"\n🔧 Preprocessing Components:")
print(f"   1. Spatial Remapping (EASYMORE)")
print(f"   2. Zonal Statistics (rasterstats)")
print(f"   3. Temporal Alignment") 
print(f"   4. Quality Control")
print(f"   5. Format Standardization")

# Execute model-agnostic preprocessing
print(f"\n⚙️  Executing model-agnostic preprocessing...")

confluence.managers['data'].run_model_agnostic_preprocessing()

print("✅ Model-agnostic preprocessing complete")

# Expected outputs
agnostic_outputs = {
    "Basin-averaged forcing": "forcing/basin_averaged_data/",
    "HRU attribute table": "attributes/hru_attributes.csv",
    "Spatial mapping files": "forcing/spatial_mapping/",
    "Quality control reports": "forcing/qc_reports/"
}

print(f"\n📁 Model-Agnostic Products:")
for product, location in agnostic_outputs.items():
    print(f"   📦 {product}: {location}")



=== Step 3c: Model-Agnostic Preprocessing Pipeline ===
Creating standardized data products for multi-model applications...

🔧 Preprocessing Components:
   1. Spatial Remapping (EASYMORE)
   2. Zonal Statistics (rasterstats)
   3. Temporal Alignment
   4. Quality Control
   5. Format Standardization

🗺️  Spatial Remapping with EASYMORE:
   Purpose: Map gridded forcing data onto irregular HRUs
   Method: Conservative area-weighted remapping
   Benefit: Preserves mass/energy conservation
   Point-scale: Simple nearest-neighbor for single HRU

📊 Zonal Statistics for Attributes:
   Elevation: Mean, min, max within each HRU
   Slope/Aspect: Representative values for energy balance
   Soil properties: Weighted by area coverage
   Land cover: Dominant class and fractional coverage

⚙️  Executing model-agnostic preprocessing...
   → Spatial remapping of forcing data
   → Zonal statistics calculation
   → Temporal alignment and quality control
12:44:08 ● Starting model-agnostic preprocessing
12

/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/easymore/remapper.py:521: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_int.to_file(self.temp_dir+self.case_name+'_intersected_shapefile.shp') # save the intersected files
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_GRU_area' to 'S_1_GRU_ar'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_basin_name' to 'S_1_basin_'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_hru_type' to 'S_1_hru_ty'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: Ru

EASYMORE version 2.0.0 is initiated.
EASYMORE is given multiple variables for remapping but only one format and fill value. EASYMORE repeats the format and fill value for all the variables in output files
EASYMORE will remap variable  airpres  from source file to variable  airpres  in remapped netCDF file
EASYMORE will remap variable  LWRadAtm  from source file to variable  LWRadAtm  in remapped netCDF file
EASYMORE will remap variable  SWRadAtm  from source file to variable  SWRadAtm  in remapped netCDF file
EASYMORE will remap variable  pptrate  from source file to variable  pptrate  in remapped netCDF file
EASYMORE will remap variable  airtemp  from source file to variable  airtemp  in remapped netCDF file
EASYMORE will remap variable  spechum  from source file to variable  spechum  in remapped netCDF file
EASYMORE will remap variable  windspd  from source file to variable  windspd  in remapped netCDF file
EASYMORE detects that the variables from the netCDF files are identicalin dim

/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/easymore/remapper.py:521: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_int.to_file(self.temp_dir+self.case_name+'_intersected_shapefile.shp') # save the intersected files
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_GRU_area' to 'S_1_GRU_ar'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_basin_name' to 'S_1_basin_'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_hru_type' to 'S_1_hru_ty'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: Ru


/Users/darrieythorsson/compHydro/data/CONFLUENCE_data/domain_paradise/forcing/temp_easymore_efa3788e_0/paradise_ERA5_target_shapefile.shp
EASYMORE detects case 1 - regular lat/lon
/Users/darrieythorsson/compHydro/data/CONFLUENCE_data/domain_paradise/forcing/temp_easymore_efa3788e_0/paradise_ERA5_source_shapefile.shp
EASYMORE created the shapefile from the netCDF file and saved it here:
EASMORE detects that target shapefile is inside the boundary of source netCDF file  and therefore correction for longitude values -180 to 180 or 0 to 360 is not performed even if  the correction_shp_lon flag is set to True [default is True]
Ended at date and time 2025-07-18 12:44:16.306343
It took 0.076933 seconds to finish creating of the remapping file
---------------------------
------REMAPPING------
netcdf output file will be compressed at level 4
Remapping /Users/darrieythorsson/compHydro/data/CONFLUENCE_data/domain_paradise/forcing/raw_data/domain_paradise_ERA5_merged_195003.nc to /Users/darrieyth

...

/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/easymore/remapper.py:521: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_int.to_file(self.temp_dir+self.case_name+'_intersected_shapefile.shp') # save the intersected files
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_GRU_area' to 'S_1_GRU_ar'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_basin_name' to 'S_1_basin_'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_hru_type' to 'S_1_hru_ty'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: Ru


/Users/darrieythorsson/compHydro/data/CONFLUENCE_data/domain_paradise/forcing/temp_easymore_57e8c595_0/paradise_ERA5_target_shapefile.shp
EASYMORE detects case 1 - regular lat/lon
/Users/darrieythorsson/compHydro/data/CONFLUENCE_data/domain_paradise/forcing/temp_easymore_57e8c595_0/paradise_ERA5_source_shapefile.shp
EASYMORE created the shapefile from the netCDF file and saved it here:
EASMORE detects that target shapefile is inside the boundary of source netCDF file  and therefore correction for longitude values -180 to 180 or 0 to 360 is not performed even if  the correction_shp_lon flag is set to True [default is True]
Ended at date and time 2025-07-18 13:21:31.893505
It took 0.058213 seconds to finish creating of the remapping file
---------------------------
------REMAPPING------
netcdf output file will be compressed at level 4
Remapping /Users/darrieythorsson/compHydro/data/CONFLUENCE_data/domain_paradise/forcing/raw_data/domain_paradise_ERA5_merged_199601.nc to /Users/darrieyth

/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/easymore/remapper.py:521: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_int.to_file(self.temp_dir+self.case_name+'_intersected_shapefile.shp') # save the intersected files
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_GRU_area' to 'S_1_GRU_ar'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_basin_name' to 'S_1_basin_'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_hru_type' to 'S_1_hru_ty'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: Ru

 SWRadAtm  in remapped netCDF file
EASYMORE will remap variable  pptrate  from source file to variable  pptrate  in remapped netCDF file
EASYMORE will remap variable  airtemp  from source file to variable  airtemp  in remapped netCDF file
EASYMORE will remap variable  spechum  from source file to variable  spechum  in remapped netCDF file
EASYMORE will remap variable  windspd  from source file to variable  windspd  in remapped netCDF file
EASYMORE detects that the variables from the netCDF files are identicalin dimensions of the variables and latitude and longitude
EASYMORE detects that all the variables have dimensions of:
['time', 'latitude', 'longitude']
EASYMORE detects that the longitude variables has dimensions of:
['longitude']
EASYMORE detects that the latitude variables has dimensions of:
['latitude']
--CREATING-REMAPPING-FILE--
Started at date and time 2025-07-18 13:21:47.339136
EASYMORE detects that target shapefile is in WGS84 (epsg:4326)
EASYMORE detects that the field for

/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/easymore/remapper.py:521: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_int.to_file(self.temp_dir+self.case_name+'_intersected_shapefile.shp') # save the intersected files
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_GRU_area' to 'S_1_GRU_ar'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_basin_name' to 'S_1_basin_'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'S_1_hru_type' to 'S_1_hru_ty'
  ogr_write(
/Users/darrieythorsson/compHydro/code/CWARHM/scienv/lib/python3.12/site-packages/pyogrio/raw.py:709: Ru

## Step 3d: Model-Specific Preprocessing
Model-specific preprocessing translates the standardized model-agnostic products into the formats and configurations required by individual models. This stage maintains the scientific benefits of standardized inputs while accommodating diverse model requirements.

Remapping of the forcing data and zonal statistics calculations for the geospatial attributes is performed in one model-agnostic pre-processing step. 

In [ ]:
# =============================================================================
# MODEL-SPECIFIC PREPROCESSING
# =============================================================================

print("\n=== Step 3d: Model-Specific Preprocessing ===")
print("Translating standardized products to model-specific formats...")

print(f"\n🎯 Target Model Configuration:")
print(f"   Model: {config_dict['HYDROLOGICAL_MODEL']}")

# Execute model-specific preprocessing
print(f"\n🔧 Executing SUMMA-specific preprocessing...")

confluence.managers['model'].preprocess_models()

print("✅ Model-specific preprocessing complete")


# Step 4: Model Instantiation and Process-Based Simulation
## Scientific Context
Model instantiation represents the critical transition from static data preparation to dynamic process simulation. This step transforms spatially-distributed inputs and temporally-varying forcing into evolving hydrological states through the explicit representation of physical processes.
In process-based hydrological modeling, we solve coupled differential equations representing:

- Energy Balance: Net radiation partitioning between sensible, latent, and ground heat fluxes
- Water Balance: Precipitation partitioning among interception, infiltration, evapotranspiration, and runoff
- Snow Physics: Accumulation, metamorphism, and melt processes with explicit energy considerations
- Soil Hydrology: Infiltration, redistribution, and drainage through layered soil profiles
- Vegetation Dynamics: Canopy interception, transpiration, and phenological controls

The SUMMA (Structure for Unifying Multiple Modeling Alternatives) framework enables systematic evaluation of process representations, making it ideal for scientific hypothesis testing and model physics assessment.

## CONFLUENCE Implementation
CONFLUENCE manages model execution through several integrated components:

- Workflow Orchestration: Automated sequencing of model initialization, spinup, and main simulation
- Configuration Management: Translation of scientific decisions into model-specific control files
- Execution Monitoring: Real-time tracking of model progress and error detection
- Output Organization: Systematic storage and cataloging of simulation results
- Quality Assurance: Automated checks for mass balance closure and physical realism

This framework ensures that model execution is reproducible, traceable, and scientifically rigorous, while handling the computational complexity behind the scenes.

In [ ]:
# =============================================================================
# STEP 4: MODEL INSTANTIATION AND PROCESS-BASED SIMULATION
# =============================================================================

print("=== Step 4: Model Instantiation and Process-Based Simulation ===")

# Display model configuration
print(f"\n⚙️  Model Execution Configuration:")
print(f"   Model: {config_dict['HYDROLOGICAL_MODEL']}")
print(f"   Executable: {config_dict.get('SUMMA_EXE', 'summa.exe')}")
print(f"   Domain: {config_dict['DOMAIN_NAME']} (1 HRU)")
print(f"   Simulation period: {config_dict['EXPERIMENT_TIME_START']} to {config_dict['EXPERIMENT_TIME_END']}")
print(f"   Spinup period: {config_dict.get('SPINUP_PERIOD', 'Not specified')}")

# Run models
print(f"\nRunning {confluence.config['HYDROLOGICAL_MODEL']} for point-scale simulation...")
confluence.managers['model'].run_models()

print("\nPoint-scale model run complete")

# Step 5: Model Evaluation and Process Validation
## Scientific Context
Model evaluation represents the critical test of whether our process-based simulation captures the essential physics of the real-world system. Effective evaluation requires:

- Multi-Variable Assessment: Testing multiple aspects of the hydrological system to avoid equifinality and ensure robust process representation
- Temporal Pattern Analysis: Evaluating both magnitude and timing of hydrological responses across seasonal cycles
- Process-Specific Metrics: Using evaluation criteria that reflect the underlying physics being tested
- Uncertainty Quantification: Understanding both observational and model uncertainty in performance assessment

For point-scale modeling, we focus on direct process validation where observations closely match the spatial and temporal scales of model representation. The Paradise SNOTEL station provides exceptional validation opportunities with co-located snow water equivalent and multi-depth soil moisture observations.

## CONFLUENCE Implementation
CONFLUENCE's evaluation framework emphasizes scientific interpretation over simple statistical metrics:

- Physically-Informed Metrics: Performance measures that reflect process understanding
- Multi-Scale Temporal Analysis: Evaluation across daily, seasonal, and inter-annual time scales
- Process Attribution: Linking performance to specific model physics and parameter values
- Comparative Context: Benchmarking against observational uncertainty and simple baselines

In [ ]:
# =============================================================================
# STEP 5: MODEL EVALUATION AND PROCESS VALIDATION
# =============================================================================

print("=== Step 5: Model Evaluation and Process Validation ===")
print("Comprehensive assessment of process-based simulation performance")

print(f"\n🎯 Evaluation Philosophy:")
print(f"   → Multi-variable validation prevents equifinality")
print(f"   → Process-specific metrics reflect underlying physics")
print(f"   → Temporal patterns as important as magnitudes")
print(f"   → Scientific interpretation guides performance assessment")

# Load and explore simulation outputs
print(f"\n📊 Simulation Output Exploration:")
sim_dir = confluence.project_dir / "simulations" / config_dict['EXPERIMENT_ID'] / "SUMMA"

expected_outputs = [
    f"{config_dict['EXPERIMENT_ID']}_day.nc",
    f"{config_dict['EXPERIMENT_ID']}_timestep.nc"
]

print(f"   Simulation directory: {sim_dir}")
for output_file in expected_outputs:
    file_path = sim_dir / output_file
    exists = "✅" if file_path.exists() else "❌"
    if file_path.exists():
        file_size = file_path.stat().st_size / (1024*1024)  # MB
        print(f"   {exists} {output_file} ({file_size:.1f} MB)")
    else:
        print(f"   {exists} {output_file}")

## Simulation Data Summary and Variable Inventory
Before diving into specific evaluations, let's comprehensively explore what our simulation produced and establish the temporal and variable scope for evaluation.

In [ ]:
# =============================================================================
# SIMULATION DATA SUMMARY AND VARIABLE INVENTORY
# =============================================================================

print("\n=== Simulation Data Summary and Variable Inventory ===")
print("Exploring simulation outputs to understand available data...")

# Load the daily output file (primary evaluation dataset)
daily_output_path = sim_dir / f"{config_dict['EXPERIMENT_ID']}_day.nc"

if daily_output_path.exists():
    import xarray as xr
    import numpy as np
    
    print(f"\n📁 Loading daily simulation data: {daily_output_path.name}")
    ds = xr.open_dataset(daily_output_path)
    
    # Basic dataset information
    print(f"\n📅 Temporal Coverage:")
    print(f"   Start date: {ds.time.min().values}")
    print(f"   End date: {ds.time.max().values}")
    print(f"   Total days: {len(ds.time)}")
    print(f"   Time step: {ds.time.diff('time')[0].values}")
    
    # Spatial dimensions
    print(f"\n🌐 Spatial Dimensions:")
    for dim, size in ds.sizes.items():
        if dim != 'time':
            print(f"   {dim}: {size}")
    
    # Variable inventory
    print(f"\n📊 Variable Inventory ({len(ds.data_vars)} variables):")
    
    # Categorize variables by process
    variable_categories = {
        'Snow Variables': ['scalarSWE', 'scalarSnowDepth', 'scalarSnowAlbedo', 'mLayerTemp', 'mLayerVolFracIce'],
        'Soil Moisture': ['mLayerVolFracLiq', 'mLayerVolFracTot', 'mLayerMatricHead'],
        'Energy Balance': ['scalarLatHeatTotal', 'scalarSenHeatTotal', 'scalarGroundHeatFlux', 'scalarNetRadiation'],
        'Water Fluxes': ['scalarRainPlusMelt', 'scalarInfiltration', 'scalarSurfaceRunoff', 'scalarCanopyTranspiration'],
        'Temperature': ['scalarCanopyTemp', 'scalarGroundTemp', 'mLayerTemp'],
        'Vegetation': ['scalarCanopyLiq', 'scalarCanopyIce', 'scalarCanopyTranspiration']
    }
    
    for category, var_list in variable_categories.items():
        available_vars = [var for var in var_list if var in ds.data_vars]
        if available_vars:
            print(f"\n   {category}:")
            for var in available_vars:
                # Get basic statistics
                var_data = ds[var]
                if var_data.ndim > 1:  # Multi-dimensional variable
                    print(f"     📈 {var} (shape: {var_data.shape[1:]})")
                else:  # 1D time series
                    var_min = float(var_data.min())
                    var_max = float(var_data.max())
                    var_mean = float(var_data.mean())
                    print(f"     📈 {var}: {var_min:.3f} to {var_max:.3f} (mean: {var_mean:.3f})")
    
    # Skip spinup period for evaluation
    start_year = ds.time.dt.year.min().values + 1
    spinup_end = f"{start_year}-01-01"
    print(f"\n🔄 Spinup Handling:")
    print(f"   Simulation start: {ds.time.min().values}")
    print(f"   Evaluation start: {spinup_end} (skipping {start_year - ds.time.dt.year.min().values} year spinup)")
    
    # Filter data after spinup for evaluation
    time_mask = ds.time >= pd.to_datetime(spinup_end)
    ds_eval = ds.isel(time=time_mask)
    
    print(f"   Evaluation period: {ds_eval.time.min().values} to {ds_eval.time.max().values}")
    print(f"   Evaluation days: {len(ds_eval.time)}")
    
    # Key statistics for primary evaluation variables
    print(f"\n🎯 Primary Evaluation Variables Summary:")
    
    if 'scalarSWE' in ds_eval.data_vars:
        swe = ds_eval['scalarSWE']
        swe_stats = {
            'max': float(swe.max()),
            'mean_winter': float(swe.where(swe.time.dt.month.isin([12,1,2,3,4,5])).mean()),
            'peak_date': swe.time[swe.argmax()].values
        }
        print(f"   ❄️  Snow Water Equivalent:")
        print(f"       Max SWE: {swe_stats['max']:.1f} mm")
        print(f"       Winter mean: {swe_stats['mean_winter']:.1f} mm")
        print(f"       Typical peak: {pd.to_datetime(swe_stats['peak_date']).strftime('%B %d')}")
    
    if 'mLayerVolFracLiq' in ds_eval.data_vars:
        sm = ds_eval['mLayerVolFracLiq']
        # Focus on near-surface layers
        if sm.sizes['midToto'] >= 3:
            sm_surface = sm.isel(midToto=slice(0,3))  # Top 3 layers
            sm_stats = {
                'mean': float(sm_surface.mean()),
                'range': float(sm_surface.max() - sm_surface.min()),
                'seasonal_var': float(sm_surface.groupby('time.month').mean().std())
            }
            print(f"   🌱 Soil Moisture (top 3 layers):")
            print(f"       Mean: {sm_stats['mean']:.3f} m³/m³")
            print(f"       Range: {sm_stats['range']:.3f} m³/m³")
            print(f"       Seasonal variability: {sm_stats['seasonal_var']:.3f} m³/m³")
    
    # Data quality summary
    print(f"\n✅ Data Quality Summary:")
    
    # Check for missing values
    total_vars = len(ds_eval.data_vars)
    vars_with_nans = sum(1 for var in ds_eval.data_vars if ds_eval[var].isnull().any())
    print(f"   Missing values: {vars_with_nans}/{total_vars} variables contain NaN")
    
    # Check for physical realism
    physically_realistic = True
    if 'scalarSWE' in ds_eval.data_vars:
        if (ds_eval['scalarSWE'] < 0).any():
            print(f"   ❌ Negative SWE values detected")
            physically_realistic = False
    
    if 'mLayerVolFracLiq' in ds_eval.data_vars:
        sm_data = ds_eval['mLayerVolFracLiq']
        if (sm_data < 0).any() or (sm_data > 1).any():
            print(f"   ❌ Soil moisture outside [0,1] range")
            physically_realistic = False
    
    if physically_realistic:
        print(f"   ✅ All variables within physically realistic ranges")
    
    print(f"\n🚀 Ready for detailed evaluation against observations!")
    
    # Store evaluation dataset for subsequent analysis
    evaluation_data = ds_eval
    
else:
    print("❌ Daily simulation output not found")
    print("   Cannot proceed with evaluation")
    raise FileNotFoundError(f"Simulation output not found: {daily_output_path}")

## Step 5a: Snow Water Equivalent Evaluation
Snow processes represent a critical test of model physics, integrating energy balance, phase change, and accumulation/melt dynamics. SWE evaluation provides direct insight into the model's ability to represent snow physics.

In [ ]:
# =============================================================================
# STEP 5A: SNOW WATER EQUIVALENT EVALUATION
# =============================================================================

print("\n=== Step 5a: Snow Water Equivalent Evaluation ===")
print("Validating snow accumulation and melt physics against SNOTEL observations...")

print(f"\n❄️  Snow Process Evaluation Framework:")
snow_processes = [
    "Precipitation phase partitioning (rain vs. snow)",
    "Snow accumulation efficiency and density evolution",
    "Energy balance-driven snowmelt timing and rates",
    "Snow albedo feedback and metamorphism effects",
    "Seasonal snow cycle timing and magnitude"
]

for process in snow_processes:
    print(f"   🌨️  {process}")

# Load observed SWE data
obs_swe_path = confluence.project_dir / "observations" / "snow" / "snotel" / "processed" / f"{config_dict['DOMAIN_NAME']}_swe_processed.csv"

print(f"\n📊 Loading observed SWE data...")
print(f"   Source: SNOTEL Station {config_dict.get('SNOTEL_STATION', 'Unknown')}")
print(f"   File: {obs_swe_path}")

if obs_swe_path.exists():
    # Load observed data
    obs_swe = pd.read_csv(obs_swe_path, parse_dates=['Date'], dayfirst=True)
    obs_swe.set_index('Date', inplace=True)
    
    # Ensure proper datetime index
    if not isinstance(obs_swe.index, pd.DatetimeIndex):
        obs_swe.index = pd.to_datetime(obs_swe.index)
    
    print(f"   ✅ Observed data loaded")
    print(f"   Period: {obs_swe.index.min()} to {obs_swe.index.max()}")
    print(f"   Records: {len(obs_swe)} observations")
    print(f"   SWE column: {obs_swe.columns.tolist()}")
    
    # Extract simulated SWE
    sim_swe = evaluation_data['scalarSWE'].to_pandas()
    print(f"\n   ✅ Simulated data extracted")
    print(f"   Period: {sim_swe.index.min()} to {sim_swe.index.max()}")
    print(f"   Records: {len(sim_swe)} time steps")
    
    # Find common period and align data
    start_date = max(obs_swe.index.min(), sim_swe.index.min())
    end_date = min(obs_swe.index.max(), sim_swe.index.max())
    
    print(f"\n🔄 Data Alignment:")
    print(f"   Common period: {start_date} to {end_date}")
    print(f"   Duration: {(end_date - start_date).days} days")
    
    # Resample to daily and filter to common period
    obs_daily = obs_swe.resample('D').mean().loc[start_date:end_date]
    sim_daily = sim_swe.resample('D').mean().loc[start_date:end_date]
    
    # Handle different column names for SWE
    if 'SWE' in obs_daily.columns:
        obs_values = obs_daily['SWE']
    elif 'swe' in obs_daily.columns:
        obs_values = obs_daily['swe']
    else:
        # Use first column
        obs_values = obs_daily.iloc[:, 0]
        print(f"   Using column '{obs_daily.columns[0]}' for observed SWE")
    
    # Remove NaN values for metrics calculation
    valid_mask = ~(obs_values.isna() | sim_daily.isna())
    obs_valid = obs_values[valid_mask]
    sim_valid = sim_daily[valid_mask]
    
    print(f"   Valid paired observations: {len(obs_valid)} days")
    
    # Calculate comprehensive performance metrics
    print(f"\n📊 Snow Water Equivalent Performance Metrics:")
    
    # Basic statistics
    rmse = np.sqrt(((obs_valid - sim_valid) ** 2).mean())
    bias = (sim_valid - obs_valid).mean()
    mae = np.abs(obs_valid - sim_valid).mean()
    corr = obs_valid.corr(sim_valid)
    
    # Percent bias
    pbias = 100 * bias / obs_valid.mean()
    
    # Nash-Sutcliffe Efficiency
    nse = 1 - ((obs_valid - sim_valid) ** 2).sum() / ((obs_valid - obs_valid.mean()) ** 2).sum()
    
    # Kling-Gupta Efficiency
    kge_corr = obs_valid.corr(sim_valid)
    kge_bias = sim_valid.mean() / obs_valid.mean()
    kge_var = (sim_valid.std() / obs_valid.std())
    kge = 1 - np.sqrt((kge_corr - 1)**2 + (kge_bias - 1)**2 + (kge_var - 1)**2)
    
    # Display metrics
    print(f"   📈 Root Mean Square Error (RMSE): {rmse:.2f} mm")
    print(f"   📈 Mean Absolute Error (MAE): {mae:.2f} mm")
    print(f"   📈 Bias: {bias:+.2f} mm ({pbias:+.1f}%)")
    print(f"   📈 Correlation: {corr:.3f}")
    print(f"   📈 Nash-Sutcliffe Efficiency: {nse:.3f}")
    print(f"   📈 Kling-Gupta Efficiency: {kge:.3f}")
    
    # Snow-specific metrics
    print(f"\n❄️  Snow-Specific Performance Assessment:")
    
    # Peak SWE analysis
    obs_peak = obs_valid.max()
    sim_peak = sim_valid.max()
    peak_bias = sim_peak - obs_peak
    peak_pbias = 100 * peak_bias / obs_peak
    
    print(f"   🏔️  Peak SWE:")
    print(f"       Observed: {obs_peak:.1f} mm")
    print(f"       Simulated: {sim_peak:.1f} mm")
    print(f"       Bias: {peak_bias:+.1f} mm ({peak_pbias:+.1f}%)")
    
    # Snow season timing
    obs_peak_date = obs_valid.idxmax()
    sim_peak_date = sim_valid.idxmax()
    timing_diff = (sim_peak_date - obs_peak_date).days
    
    print(f"   📅 Peak Timing:")
    print(f"       Observed peak: {obs_peak_date.strftime('%B %d, %Y')}")
    print(f"       Simulated peak: {sim_peak_date.strftime('%B %d, %Y')}")
    print(f"       Timing difference: {timing_diff:+d} days")
    
    # Snow season length
    snow_threshold = 10  # mm
    obs_snow_days = (obs_valid > snow_threshold).sum()
    sim_snow_days = (sim_valid > snow_threshold).sum()
    
    print(f"   ⏰ Snow Season (SWE > {snow_threshold} mm):")
    print(f"       Observed: {obs_snow_days} days")
    print(f"       Simulated: {sim_snow_days} days")
    print(f"       Difference: {sim_snow_days - obs_snow_days:+d} days")
    
    # Create comprehensive visualization
    print(f"\n📈 Creating SWE comparison visualization...")
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Time series plot
    ax1 = axes[0, 0]
    ax1.plot(obs_daily.index, obs_values, 'o-', label='Observed', 
             color='black', alpha=0.7, markersize=3, linewidth=1)
    ax1.plot(sim_daily.index, sim_daily, '-', label='Simulated', 
             color='blue', linewidth=2)
    ax1.set_title('Snow Water Equivalent Time Series', fontsize=12, fontweight='bold')
    ax1.set_ylabel('SWE (mm)', fontsize=11)
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Add performance metrics as text
    metrics_text = f'RMSE: {rmse:.1f} mm\nBias: {bias:+.1f} mm\nCorr: {corr:.3f}\nNSE: {nse:.3f}'
    ax1.text(0.02, 0.95, metrics_text, transform=ax1.transAxes, 
             bbox=dict(facecolor='white', alpha=0.8), fontsize=10, verticalalignment='top')
    
    # Scatter plot
    ax2 = axes[0, 1]
    ax2.scatter(obs_valid, sim_valid, alpha=0.6, c='blue', s=20)
    max_val = max(obs_valid.max(), sim_valid.max())
    ax2.plot([0, max_val], [0, max_val], 'k--', label='1:1 line')
    ax2.set_xlabel('Observed SWE (mm)', fontsize=11)
    ax2.set_ylabel('Simulated SWE (mm)', fontsize=11)
    ax2.set_title('Observed vs. Simulated SWE', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    ax2.set_aspect('equal', adjustable='box')
    
    # Seasonal cycle
    ax3 = axes[1, 0]
    obs_monthly = obs_values.groupby(obs_values.index.month).mean()
    sim_monthly = sim_daily.groupby(sim_daily.index.month).mean()
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    ax3.plot(range(1, 13), obs_monthly, 'o-', label='Observed', color='black', linewidth=2)
    ax3.plot(range(1, 13), sim_monthly, 'o-', label='Simulated', color='blue', linewidth=2)
    ax3.set_xticks(range(1, 13))
    ax3.set_xticklabels(months, rotation=45)
    ax3.set_ylabel('Mean SWE (mm)', fontsize=11)
    ax3.set_title('Seasonal Cycle', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.legend()
    
    # Residuals over time
    ax4 = axes[1, 1]
    residuals = sim_valid - obs_valid
    ax4.scatter(obs_valid.index, residuals, alpha=0.6, c='red', s=15)
    ax4.axhline(y=0, color='black', linestyle='-', alpha=0.5)
    ax4.axhline(y=residuals.std(), color='red', linestyle='--', alpha=0.5, label=f'±1σ ({residuals.std():.1f} mm)')
    ax4.axhline(y=-residuals.std(), color='red', linestyle='--', alpha=0.5)
    ax4.set_ylabel('Residuals (Sim - Obs) [mm]', fontsize=11)
    ax4.set_title('Model Residuals Over Time', fontsize=12, fontweight='bold')
    ax4.grid(True, alpha=0.3)
    ax4.legend()
    
    plt.suptitle(f'Snow Water Equivalent Evaluation - {config_dict["DOMAIN_NAME"].title()}', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Scientific interpretation
    print(f"\n🔬 Scientific Interpretation of SWE Performance:")
    
    # Performance classification
    if nse > 0.7 and abs(pbias) < 15:
        performance = "Excellent"
    elif nse > 0.5 and abs(pbias) < 25:
        performance = "Good" 
    elif nse > 0.3 and abs(pbias) < 35:
        performance = "Satisfactory"
    else:
        performance = "Needs improvement"
    
    print(f"   🎯 Overall Performance: {performance}")
    
    # Process-specific insights
    if abs(timing_diff) <= 10:
        print(f"   ✅ Excellent snow melt timing (within {abs(timing_diff)} days)")
    elif abs(timing_diff) <= 20:
        print(f"   ⚠️  Moderate timing bias ({timing_diff:+d} days)")
    else:
        print(f"   ❌ Significant timing bias ({timing_diff:+d} days)")
    
    if abs(peak_pbias) <= 15:
        print(f"   ✅ Good peak SWE representation ({peak_pbias:+.1f}% bias)")
    else:
        print(f"   ⚠️  Peak SWE bias requires attention ({peak_pbias:+.1f}%)")
    
    if corr > 0.8:
        print(f"   ✅ Strong temporal correlation (r = {corr:.3f})")
    elif corr > 0.6:
        print(f"   ⚠️  Moderate temporal correlation (r = {corr:.3f})")
    else:
        print(f"   ❌ Weak temporal correlation (r = {corr:.3f})")

else:
    print(f"❌ Observed SWE data not found at {obs_swe_path}")
    print("   Cannot perform SWE evaluation")
    print("   Check SNOTEL data processing step")

## Step 5b: Soil Moisture Profile Evaluation
Soil moisture evaluation tests the model's representation of vadose zone processes, including infiltration, drainage, and vertical redistribution. Multi-depth observations provide unprecedented validation opportunities for soil physics.

In [ ]:
# =============================================================================
# STEP 5B: SOIL MOISTURE PROFILE EVALUATION
# =============================================================================

print("\n=== Step 5b: Soil Moisture Profile Evaluation ===")
print("Validating soil hydrology and vadose zone processes...")

print(f"\n🌱 Soil Process Evaluation Framework:")
soil_processes = [
    "Infiltration and surface water partitioning",
    "Vertical water redistribution through soil profile", 
    "Evapotranspiration from different soil depths",
    "Soil moisture memory and persistence",
    "Seasonal soil moisture dynamics and freeze-thaw"
]

for process in soil_processes:
    print(f"   🏔️  {process}")

# Load observed soil moisture data
obs_sm_path = confluence.project_dir / "observations" / "soil_moisture" / "ismn" / "processed" / f"{config_dict['DOMAIN_NAME']}_sm_processed.csv"

print(f"\n📊 Loading observed soil moisture data...")
print(f"   Source: ISMN/SNOTEL soil moisture sensors")
print(f"   File: {obs_sm_path}")

if obs_sm_path.exists():
    # Load observed data
    obs_sm = pd.read_csv(obs_sm_path, parse_dates=['timestamp'])
    obs_sm.set_index('timestamp', inplace=True)
    
    # Ensure proper datetime index
    if not isinstance(obs_sm.index, pd.DatetimeIndex):
        obs_sm.index = pd.to_datetime(obs_sm.index)
    
    print(f"   ✅ Observed data loaded")
    print(f"   Period: {obs_sm.index.min()} to {obs_sm.index.max()}")
    print(f"   Records: {len(obs_sm)} observations")
    
    # Identify observed depth columns
    obs_depth_cols = [col for col in obs_sm.columns if col.startswith('sm_')]
    print(f"   Available depths: {obs_depth_cols}")
    
    # Extract observed depth values
    obs_depths = []
    for depth_col in obs_depth_cols:
        # Extract depth from column name (e.g., 'sm_0.0508_0.0508' -> 0.0508)
        depth_str = depth_col.split('_')[1]
        obs_depths.append(float(depth_str))
    
    print(f"   Observation depths: {[f'{d:.4f}m' for d in obs_depths]}")
    
    # Extract simulated soil moisture
    if 'mLayerVolFracLiq' in evaluation_data.data_vars:
        sim_sm = evaluation_data['mLayerVolFracLiq']
        sim_depths = evaluation_data['mLayerDepth']
        
        print(f"\n   ✅ Simulated data extracted")
        print(f"   Simulation layers: {sim_sm.sizes['midToto']}")
        print(f"   Period: {sim_sm.time.min().values} to {sim_sm.time.max().values}")
        
        # Calculate representative layer depths
        mean_layer_depths = sim_depths.mean(dim='time')
        valid_layers = mean_layer_depths > 0  # Filter out invalid layers
        
        print(f"   Valid simulation layers: {valid_layers.sum().values}")
        if valid_layers.sum() > 0:
            valid_depths_values = mean_layer_depths.where(valid_layers).dropna(dim='midToto')
            print(f"   Simulation depths: {[f'{d:.4f}m' for d in valid_depths_values.values]}")
        
        # Find common period
        start_date = max(obs_sm.index.min(), pd.to_datetime(sim_sm.time.min().values))
        end_date = min(obs_sm.index.max(), pd.to_datetime(sim_sm.time.max().values))
        
        print(f"\n🔄 Data Alignment:")
        print(f"   Common period: {start_date} to {end_date}")
        print(f"   Duration: {(end_date - start_date).days} days")
        
        # Filter to common period
        obs_period = obs_sm.loc[start_date:end_date]
        sim_time_mask = (sim_sm.time >= start_date) & (sim_sm.time <= end_date)
        sim_period = sim_sm.isel(time=sim_time_mask)
        sim_depths_period = sim_depths.isel(time=sim_time_mask)
        
        # Create comprehensive multi-depth evaluation
        print(f"\n📊 Multi-Depth Soil Moisture Performance:")
        
        n_depths = len(obs_depth_cols)
        depth_results = {}
        
        # Analyze each observed depth
        for i, (depth_col, obs_depth) in enumerate(zip(obs_depth_cols, obs_depths)):
            print(f"\n   🎯 Depth {i+1}: {obs_depth:.4f}m ({depth_col})")
            
            # Find closest simulated layer
            mean_depths = sim_depths_period.mean(dim='time')
            valid_mask = mean_depths > 0
            
            if valid_mask.sum() > 0:
                valid_mean_depths = mean_depths.where(valid_mask)
                depth_differences = np.abs(valid_mean_depths - obs_depth)
                closest_layer_idx = depth_differences.argmin().values
                closest_layer_depth = valid_mean_depths[closest_layer_idx].values
                
                print(f"       Matched to simulation layer {closest_layer_idx} (depth: {closest_layer_depth:.4f}m)")
                
                # Extract data for this layer
                obs_layer = obs_period[depth_col]
                sim_layer = sim_period.isel(midToto=closest_layer_idx, hru=0)
                
                # Convert to pandas for easier handling
                sim_layer_ts = sim_layer.to_pandas()
                
                # Resample to daily and align
                obs_daily = obs_layer.resample('D').mean()
                sim_daily = sim_layer_ts.resample('D').mean()
                
                # Remove invalid values (negative soil moisture indicates missing data)
                sim_daily = sim_daily.where(sim_daily > -100)
                
                # Find valid paired data
                valid_data_mask = ~(obs_daily.isna() | sim_daily.isna())
                obs_valid = obs_daily[valid_data_mask]
                sim_valid = sim_daily[valid_data_mask]
                
                if len(obs_valid) > 10:  # Require minimum data for meaningful evaluation
                    # Calculate performance metrics
                    rmse = np.sqrt(((obs_valid - sim_valid) ** 2).mean())
                    bias = (sim_valid - obs_valid).mean()
                    mae = np.abs(obs_valid - sim_valid).mean()
                    corr = obs_valid.corr(sim_valid)
                    
                    # Store results
                    depth_results[obs_depth] = {
                        'obs_valid': obs_valid,
                        'sim_valid': sim_valid,
                        'rmse': rmse,
                        'bias': bias,
                        'mae': mae,
                        'corr': corr,
                        'layer_idx': closest_layer_idx,
                        'sim_depth': closest_layer_depth
                    }
                    
                    print(f"       📈 RMSE: {rmse:.3f} m³/m³")
                    print(f"       📈 Bias: {bias:+.3f} m³/m³")
                    print(f"       📈 Correlation: {corr:.3f}")
                    print(f"       📈 Valid pairs: {len(obs_valid)}")
                    
                else:
                    print(f"       ⚠️  Insufficient valid data pairs ({len(obs_valid)})")
            else:
                print(f"       ❌ No valid simulation layers found")
        
        # Create comprehensive visualization
        if depth_results:
            print(f"\n📈 Creating soil moisture profile evaluation...")
            
            n_depths = len(depth_results)
            fig, axes = plt.subplots(n_depths, 2, figsize=(15, 4*n_depths))
            
            if n_depths == 1:
                axes = axes.reshape(1, -1)
            
            for i, (obs_depth, results) in enumerate(depth_results.items()):
                # Time series plot
                ax_ts = axes[i, 0]
                ax_ts.plot(results['obs_valid'].index, results['obs_valid'], 'o-', 
                          label=f'Observed ({obs_depth:.4f}m)', 
                          color='black', alpha=0.7, markersize=2, linewidth=1)
                ax_ts.plot(results['sim_valid'].index, results['sim_valid'], '-', 
                          label=f'Simulated (L{results["layer_idx"]}, {results["sim_depth"]:.4f}m)', 
                          color='blue', linewidth=2)
                
                ax_ts.set_title(f'Soil Moisture at {obs_depth:.4f}m depth', fontsize=11, fontweight='bold')
                ax_ts.set_ylabel('Soil Moisture (m³/m³)', fontsize=10)
                ax_ts.grid(True, alpha=0.3)
                ax_ts.legend(fontsize=9)
                
                # Add metrics
                metrics_text = (f"RMSE: {results['rmse']:.3f}\n"
                               f"Bias: {results['bias']:+.3f}\n"
                               f"Corr: {results['corr']:.3f}")
                ax_ts.text(0.02, 0.95, metrics_text, transform=ax_ts.transAxes,
                          bbox=dict(facecolor='white', alpha=0.8), fontsize=9, verticalalignment='top')
                
                # Scatter plot
                ax_scatter = axes[i, 1]
                ax_scatter.scatter(results['obs_valid'], results['sim_valid'], 
                                  alpha=0.6, c='blue', s=15)
                
                # 1:1 line
                min_val = min(results['obs_valid'].min(), results['sim_valid'].min())
                max_val = max(results['obs_valid'].max(), results['sim_valid'].max())
                ax_scatter.plot([min_val, max_val], [min_val, max_val], 'k--', 
                               label='1:1 line', alpha=0.7)
                
                ax_scatter.set_xlabel('Observed SM (m³/m³)', fontsize=10)
                ax_scatter.set_ylabel('Simulated SM (m³/m³)', fontsize=10)
                ax_scatter.set_title(f'Obs vs Sim at {obs_depth:.4f}m', fontsize=11, fontweight='bold')
                ax_scatter.grid(True, alpha=0.3)
                ax_scatter.legend(fontsize=9)
                ax_scatter.set_aspect('equal', adjustable='box')
            
            plt.suptitle(f'Soil Moisture Profile Evaluation - {config_dict["DOMAIN_NAME"].title()}', 
                         fontsize=14, fontweight='bold')
            plt.tight_layout()
            plt.show()
            
            # Summary performance assessment
            print(f"\n🔬 Soil Moisture Profile Performance Summary:")
            
            avg_rmse = np.mean([r['rmse'] for r in depth_results.values()])
            avg_corr = np.mean([r['corr'] for r in depth_results.values()])
            avg_bias = np.mean([r['bias'] for r in depth_results.values()])
            
            print(f"   📊 Profile-averaged metrics:")
            print(f"       Average RMSE: {avg_rmse:.3f} m³/m³")
            print(f"       Average correlation: {avg_corr:.3f}")
            print(f"       Average bias: {avg_bias:+.3f} m³/m³")
            
            # Depth-dependent performance analysis
            print(f"\n   🌱 Depth-Dependent Performance:")
            for obs_depth, results in depth_results.items():
                if results['corr'] > 0.6 and results['rmse'] < 0.1:
                    performance = "Good"
                elif results['corr'] > 0.4 and results['rmse'] < 0.15:
                    performance = "Fair"
                else:
                    performance = "Poor"
                
                print(f"       {obs_depth:.4f}m: {performance} (r={results['corr']:.3f}, RMSE={results['rmse']:.3f})")
            
            # Scientific interpretation
            print(f"\n🎯 Scientific Interpretation:")
            
            if avg_corr > 0.6:
                print(f"   ✅ Model captures temporal dynamics well")
            else:
                print(f"   ⚠️  Temporal dynamics need improvement")
            
            if abs(avg_bias) < 0.05:
                print(f"   ✅ Minimal systematic bias in soil moisture")
            else:
                print(f"   ⚠️  Systematic bias detected ({avg_bias:+.3f} m³/m³)")
            
            if avg_rmse < 0.1:
                print(f"   ✅ Reasonable magnitude errors")
            else:
                print(f"   ⚠️  Large magnitude errors suggest parameter issues")
            
            # Process insights
            depth_performance = {depth: results['corr'] for depth, results in depth_results.items()}
            best_depth = max(depth_performance, key=depth_performance.get)
            worst_depth = min(depth_performance, key=depth_performance.get)
            
            print(f"\n   🔍 Process Insights:")
            print(f"       Best performance: {best_depth:.4f}m (r={depth_performance[best_depth]:.3f})")
            print(f"       Worst performance: {worst_depth:.4f}m (r={depth_performance[worst_depth]:.3f})")
            
            if best_depth < 0.1:
                print(f"       → Surface processes well represented")
            elif best_depth > 0.5:
                print(f"       → Deep soil processes well represented")
            
        else:
            print(f"   ❌ No valid depth comparisons possible")
    
    else:
        print(f"   ❌ Simulated soil moisture data not found in outputs")

else:
    print(f"❌ Observed soil moisture data not found at {obs_sm_path}")
    print("   Cannot perform soil moisture evaluation")
    print("   Check soil moisture data processing step")

# Close evaluation dataset
evaluation_data.close()

In [ ]:
# Tutorial Summary and Next Steps

## Summary: Point-Scale Snow and Soil Process Validation

This tutorial demonstrated the complete CONFLUENCE workflow for point-scale hydrological modeling, establishing fundamental principles for reproducible computational hydrology research.

### 🎯 Key Achievements

- **✅ Reproducible Workflow Management**: Configuration-driven experiments with complete provenance tracking
- **✅ Model-Agnostic Preprocessing**: Standardized data pipeline enabling true model physics comparisons  
- **✅ Process-Based Simulation**: Explicit energy and water balance with SUMMA's modular physics
- **✅ Multi-Variable Validation**: Co-located snow water equivalent and multi-depth soil moisture evaluation
- **✅ Scientific Interpretation**: Performance assessment linked to underlying physical processes

### 🔬 Scientific Process Validation

- **Snow Physics**: Accumulation, metamorphism, and energy-balance driven melt processes
- **Soil Hydrology**: Multi-depth moisture dynamics and vertical water redistribution
- **Energy Balance**: Surface-atmosphere energy exchange and flux partitioning
- **Temporal Dynamics**: Daily to seasonal process evolution and memory effects
- **Physical Realism**: Mass and energy conservation with realistic state variable bounds

### 🏗️ CONFLUENCE Framework Capabilities

- **Modular Architecture**: Specialized managers for different workflow components
- **Workflow Orchestration**: Automated step sequencing with restart capabilities
- **Quality Assurance**: Built-in validation and error checking throughout pipeline
- **Scalable Design**: Same framework supports point-scale through continental applications
- **Research Continuity**: Foundation for distributed modeling and large-sample studies

---

## Looking Ahead: Tutorial 01b - Energy Flux Validation at FLUXNET Sites

### 🌿 **Next Focus: Evapotranspiration and Energy Balance Processes**

While Tutorial 01a focused on **snow and soil moisture dynamics** in a mountain environment, Tutorial 01b expands our process validation to **energy flux partitioning and evapotranspiration** using FLUXNET observations.

### **Scientific Context for Tutorial 01b**

**FLUXNET towers** provide exceptional validation opportunities for energy balance modeling through direct eddy covariance measurements of:

- **Latent Heat Flux (LE)**: Direct measurement of evapotranspiration energy
- **Sensible Heat Flux (H)**: Convective energy transfer to atmosphere  
- **Net Radiation (Rn)**: Available energy driving surface processes
- **Ground Heat Flux (G)**: Soil energy storage and conduction
- **Carbon Fluxes (NEE)**: Ecosystem productivity and respiration

### **Tutorial 01b Objectives**

1. **Energy Balance Closure**: Validate SUMMA's energy partitioning against tower observations
2. **ET Process Validation**: Test stomatal conductance, canopy interception, and soil evaporation
3. **Vegetation Dynamics**: Evaluate LAI, phenology, and canopy process representations
4. **Multi-Scale Temporal Analysis**: Assess performance from sub-daily to seasonal time scales
5. **Cross-Biome Validation**: Demonstrate framework across forest, grassland, and agricultural sites

### **CONFLUENCE Advantages for FLUXNET Analysis**

- **Same Preprocessing Framework**: Model-agnostic pipeline serves both snow and flux validation
- **Multi-Variable Evaluation**: Simultaneous assessment of energy, water, and carbon processes
- **Process Attribution**: Link performance to specific model physics and parameter values
- **Comparative Analysis**: Benchmarking across sites, seasons, and process representations

### **Scientific Progression**

| Tutorial | Process Focus | Validation Data | Environment | Model Complexity |
|----------|---------------|-----------------|-------------|------------------|
| **01a** | Snow & Soil Hydrology | SNOTEL (SWE, SM) | Mountain | Point-scale |
| **01b** | Energy Balance & ET | FLUXNET (LE, H, Rn) | Multiple biomes | Point-scale |
| **02** | Basin-scale Hydrology | Streamflow | Watershed | Spatial routing |
| **02** | Domain-scale Hydrology | Streamflow | Large Domain | Thousands of basins + river segments |                                                               
| **04** | Large-Sample Analysis | Multi-site | Global | Thousands of sites |

---

**Ready to explore energy flux validation?** → **[Tutorial 01b: Point-Scale Energy Balance Validation](./01b_point_scale_fluxnet.ipynb)**

The process validation foundation established here directly enables the energy flux analysis in Tutorial 01b, building toward comprehensive model evaluation across the full spectrum of hydrological processes.